# Credit Card Default Prediction - Data Sourcing

### Introduction

The goal of this project is to use publically available macroeconomic data to predict the aggregate level of credit card defaults. This notebook represents the first stage of the project: sourcing both the independent variables, which are the macroeconomic variables that may be useful in predicting the credit card default rate, and the dependent variable, the proportion of credit cards defaulting each quarter.

### Setting Up the API Calls

The first steps are to set up the notebook to make the requisite API requests. This involves importing the packages necessary for the requests and data processing, setting up the API, and creating a list of the variables that will be pulled.

First, the required packages are imported.

In [1]:
import numpy as np
import pandas as pd
import fredapi
import datetime as dt
import calendar
import os
import time

Most of the data is sourced from Federal Reserve Economic Data (FRED) through an API. The following sets the FRED API key and sets up the fredapi package with it. While API calls can be made to the FRED API using the requests package, the fredapi package was selected as it dramatically simplifies the process. FRED API keys can be sourced at the following site:

https://research.stlouisfed.org/docs/api/fred/

In [2]:
API_KEY = ''
fred = fredapi.Fred(api_key=API_KEY)

Several data sources will be pulled for this project. This step sets up a list of dictionaries, each of which contains information necessary to pull a particular data source. The components of each dictionary are:
* name: The name of the dataset. This is used to provide a more readable description for the field than the alphanumeric FRED codes that are used in the API.
* code: This is the alphanumeric string that FRED uses to identify each data series. It is used in the API calls.
* freq: Different datasets have different frequencies, including daily, weekly, monthly, quarterly, and annually. Each unique freuency requires a different processing method to make it usable in any future model, so the frequency of the series is recorded for future use.

In [3]:
s_list = [{'name': 'Ten Year Minus Two Year Yield Diff', 'code': 'T10Y2Y', 'freq': 'd'},
          {'name': 'Ten Year Minus Three Month Yield Diff', 'code': 'T10Y3M', 'freq': 'd'},
          {'name': 'CPI - All Items', 'code': 'CPIAUCSL', 'freq': 'm'},
          {'name': 'Civilian Unemployment Rate', 'code': 'UNRATE', 'freq': 'm'},
          {'name': 'US RGDP', 'code': 'GDPC1', 'freq': 'q'},
          {'name': '30-Year Fixed Mortgage Average', 'code': 'MORTGAGE30US', 'freq': 'w'},
          {'name': 'US NGDP', 'code': 'GDP', 'freq': 'q'},
          {'name': 'Effective Federal Funds Rate', 'code': 'FEDFUNDS', 'freq': 'm'},
          {'name': 'Ten Year Treasury Constant Maturity Rate', 'code': 'DGS10', 'freq': 'd'},
          {'name': 'ICE BofAML US High Yield Master II Option-Adjusted Spread', 'code': 'BAMLH0A0HYM2', 'freq': 'd'},
          {'name': 'S&P/Case-Shiller U.S. National Home Price Index', 'code': 'CSUSHPINSA', 'freq': 'm'},
          {'name': '3-Month LIBOR', 'code': 'USD3MTD156N', 'freq': 'd'},
          {'name': '3-Month Treasury Bill Secondary Rate', 'code': 'TB3MS', 'freq': 'm'},
          {'name': '1-Month LIBOR', 'code': 'USD1MTD156N', 'freq': 'd'},
          {'name': 'M2 Money Stock', 'code': 'M2', 'freq': 'w'},
          {'name': '10-Year Breakeven Inflation Rate', 'code': 'T10YIE', 'freq': 'd'},
          {'name': 'Industrial Production Index', 'code': 'INDPRO', 'freq': 'm'},
          {'name': 'TED Spread', 'code': 'TEDRATE', 'freq': 'd'},
          {'name': 'Trade-Weighted US Dollar Index', 'code': 'TWEXB', 'freq': 'w'},
          {'name': 'Federal Debt: Total Public Debt as Percent of GDP', 'code': 'GFDEGDQ188S', 'freq': 'q'},
          {'name': '1-Year Treasury Constant Maturity Rate', 'code': 'DGS1', 'freq': 'd'},
          {'name': 'All Employees: Total Nonfarm Payrolls', 'code': 'PAYEMS', 'freq': 'm'},
          {'name': 'Smoothed U.S. Recession Probabilities', 'code': 'RECPROUSM156N', 'freq': 'm'},
          {'name': 'Leading Index for the United States', 'code': 'USSLIND', 'freq': 'm'},
          {'name': 'S&P 500', 'code': 'SP500', 'freq': 'd'},
          {'name': 'Real Median Household Income in the United States', 'code': 'MEHOINUSA672N', 'freq': 'a'},
          {'name': 'Moody\'s Seasoned Baa Corporate Bond Yield', 'code': 'BAA', 'freq': 'm'},
          {'name': 'Moody\'s Seasoned Aaa Corporate Bond Yield', 'code': 'AAA', 'freq': 'm'},
          {'name': 'St. Louis Fed Financial Stress Index', 'code': 'STLFSI', 'freq': 'w'},
          {'name': 'ICE BofAML US Corporate BBB Option-Adjusted Spread', 'code': 'BAMLC0A4CBBB', 'freq': 'd'},
          {'name': 'Crude Oil Prices: West Texas Intermediate (WTI) - Cushing, Oklahoma', 'code': 'DCOILWTICO', 'freq': 'd'},
          {'name': '5-Year, 5-Year Forward Inflation Expectation Rate', 'code': 'T5YIFR', 'freq': 'd'},
          {'name': '2-Year Treasury Constant Maturity Rate', 'code': 'DGS2', 'freq': 'd'},
          {'name': 'Housing Starts: Total: New Privately Owned Housing Units Started', 'code': 'HOUST', 'freq': 'm'},
          {'name': 'U.S. / Euro Foreign Exchange Rate', 'code': 'DEXUSEU', 'freq': 'd'},
          {'name': 'S&P/Case-Shiller 20-City Composite Home Price Index', 'code': 'SPCS20RSA', 'freq': 'm'},
          {'name': 'ICE BofAML US High Yield Master II Effective Yield', 'code': 'BAMLH0A0HYM2EY', 'freq': 'd'},
          {'name': '30-Year Treasury Constant Maturity Rate', 'code': 'DGS30', 'freq': 'd'},
          {'name': 'Moody\'s Seasoned Baa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity', 'code': 'BAA10Y', 'freq': 'd'},
          {'name': 'Personal Saving Rate', 'code': 'PSAVERT', 'freq': 'm'},
          {'name': '5-Year Treasury Constant Maturity Rate', 'code': 'DGS5', 'freq': 'd'},
          {'name': 'Gold Fixing Price 10:30 A.M. (London time) in London Bullion Market, based in U.S. Dollars', 'code': 'GOLDAMGBD228NLBM', 'freq': 'd'},
          {'name': '10-Year Treasury Constant Maturity Minus Federal Funds Rate', 'code': 'T10YFF', 'freq': 'd'},
          {'name': 'Federal Surplus or Deficit', 'code': 'FYFSD', 'freq': 'a'},
          {'name': 'Median Sales Price of Houses Sold for the United States', 'code': 'MSPUS', 'freq': 'q'},
          {'name': 'Producer Price Index by Commodity for Pulp, Paper, and Allied Products: Wood Pulp', 'code': 'WPU0911', 'freq': 'q'},
          {'name': 'ICE BofAML US High Yield CCC or Below Option-Adjusted Spread', 'code': 'BAMLH0A3HYC', 'freq': 'd'},
          {'name': 'Total Vehicle Sales', 'code': 'TOTALSA', 'freq': 'm'},
          {'name': 'Initial Claims', 'code': 'ICSA', 'freq': 'w'},
          {'name': 'Interest Rates, Discount Rate for United States', 'code': 'INTDSRUSM193N', 'freq': 'm'},
          {'name': 'Civilian Labor Force Participation Rate', 'code': 'CIVPART', 'freq': 'm'},
          {'name': 'Dow Jones Industrial Average', 'code': 'DJIA', 'freq': 'd'},
          {'name': 'Real gross domestic product per capita', 'code': 'A939RX0Q048SBEA', 'freq': 'q'},
          {'name': 'Delinquency Rate on Single-Family Residential Mortgages, Booked in Domestic Offices, All Commercial Banks', 'code': 'DRSFRMACBN', 'freq': 'q'},
          {'name': 'Real Disposable Personal Income', 'code': 'DSPIC96', 'freq': 'm'},
          {'name': 'Monthly Supply of Houses in the United States', 'code': 'MSACSR', 'freq': 'm'},
          {'name': 'Employed full time: Median usual weekly real earnings: Wage and salary workers: 16 years and over', 'code': 'LES1252881600Q', 'freq': 'q'},
          {'name': 'All Employees: Manufacturing', 'code': 'MANEMP', 'freq': 'm'},
          {'name': 'Corporate Profits After Tax (without IVA and CCAdj)', 'code': 'CP', 'freq': 'q'},
          {'name': 'ICE BofAML US Corporate Master Option-Adjusted Spread', 'code': 'BAMLC0A0CM', 'freq': 'd'},
          {'name': '3-Month Treasury Constant Maturity Rate', 'code': 'DGS3MO', 'freq': 'd'},
          {'name': 'Unemployment Rate: 20 years and over', 'code': 'LNS14000024', 'freq': 'm'},
          {'name': 'All-Transactions House Price Index for the United States', 'code': 'USSTHPI', 'freq': 'q'},
          {'name': '15-Year Fixed Rate Mortgage Average in the United States', 'code': 'MORTGAGE15US', 'freq': 'w'},
          {'name': 'Commercial and Industrial Loans, All Commercial Banks', 'code': 'BUSLOANS', 'freq': 'm'},
          {'name': 'Stock Market Capitalization to GDP for United States', 'code': 'DDDM01USA156NWDB', 'freq': 'a'},
          {'name': 'Household Debt Service Payments as a Percent of Disposable Personal Income', 'code': 'TDSP', 'freq': 'q'},
          {'name': 'Bank Prime Loan Rate', 'code': 'MPRIME', 'freq': 'm'},
          {'name': 'CBOE Volatility Index: VIX', 'code': 'VIXCLS', 'freq': 'd'},
          {'name': 'NBER based Recession Indicators for the United States from the Period following the Peak through the Trough', 'code': 'USREC', 'freq': 'm'},
          {'name': 'ICE BofAML US High Yield Master II Total Return Index Value', 'code': 'BAMLHYH0A0HYM2TRIV', 'freq': 'd'},
          {'name': 'ICE BofAML US High Yield CCC or Below Effective Yield', 'code': 'BAMLH0A3HYCEY', 'freq': 'd'},
          {'name': '10-Year High Quality Market (HQM) Corporate Bond Spot Rate', 'code': 'HQMCB10YR', 'freq': 'm'},
          {'name': 'ICE BofAML US High Yield BB Option-Adjusted Spread', 'code': 'BAMLH0A1HYBB', 'freq': 'd'},
          {'name': '12-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar', 'code': 'USD12MD156N', 'freq': 'd'},
          {'name': 'Gross Private Domestic Investment', 'code': 'GPDI', 'freq': 'q'},
          {'name': 'Real Gross Private Domestic Investment', 'code': 'GPDIC1', 'freq': 'q'},
          {'name': 'Manufacturers\' New Orders: Durable Goods', 'code': 'DGORDER', 'freq': 'm'},
          {'name': 'Producer Price Index for All Commodities', 'code': 'PPIACO', 'freq': 'm'},
          {'name': 'Chicago Fed National Financial Conditions Index', 'code': 'NFCI', 'freq': 'w'},
          {'name': 'Unemployment Rate: 20 years and over, Black or African American Men', 'code': 'LNS14000031', 'freq': 'm'},
          {'name': 'Nonfinancial corporate business; debt securities; liability, Level', 'code': 'NCBDBIQ027S', 'freq': 'q'},
          {'name': 'Capacity Utilization: Total Industry', 'code': 'TCU', 'freq': 'm'},
          {'name': 'ICE BofAML US High Yield BB Effective Yield', 'code': 'BAMLH0A1HYBBEY', 'freq': 'd'},
          {'name': 'Average Sales Price of Houses Sold for the United States', 'code': 'ASPUS', 'freq': 'q'},
          {'name': 'Real Disposable Personal Income: Per Capita', 'code': 'A229RX0', 'freq': 'm'},
          {'name': 'Unemployment Level', 'code': 'UNEMPLOY', 'freq': 'm'},
          {'name': 'Homeownership Rate for the United States', 'code': 'RHORUSQ156N', 'freq': 'q'},
          {'name': 'Motor Vehicle Retail Sales: Heavy Weight Trucks', 'code': 'HTRUCKSSAAR', 'freq': 'm'},
          {'name': 'Civilian Labor Force Participation Rate: 25 to 54 years', 'code': 'LNS11300060', 'freq': 'm'},
          {'name': 'University of Michigan: Consumer Sentiment', 'code': 'UMCSENT', 'freq': 'm'},
          {'name': 'Consumer Opinion Surveys: Confidence Indicators: Composite Indicators: OECD Indicator for the United States', 'code': 'CSCICP03USM665S', 'freq': 'm'},
          {'name': 'Business Tendency Surveys for Manufacturing: Confidence Indicators: Composite Indicators: OECD Indicator for the United States', 'code': 'BSCICP03USM665S', 'freq': 'm'},
          {'name': 'Sahm Real-Time Recession Indicator', 'code': 'SAHMREALTIME', 'freq': 'm'}]

### Defining Functions for Data Manipulation

Each of the datasets defined above will be be pulled from the API and manipulated according to its frequency. In the steps below, functions are defined for how each series will be manipulated based on its frequency. Daily and weekly metrics are resampled to monthly information in a variety of ways. Quarterly and annual metrics are imputed to the monthly level.

This function defines how daily data is converted to monthly data. After the data is cleaned, a variety of steps are performed:
1. The change in the variable over a variety of periods is calculated. For example, the change from the prior day is calculated, as is the change from 365 days prior and several periods in between.
2. For each column of data - including the metric and the differences calculated above - the minimum, maximum, mean, median, and standard deviation is calculated for each month represented in the data.

In [4]:
def daily_to_monthly(series, series_name):
    """Take in a daily dataset, and convert it to a monthly df of statistics"""
    # Create a series of dates containing the BOM and EOM values of interest
    start_date = series.index.min()
    end_date = series.index.max()
    
    # Set the start date to the beginning of the earliest month in the data
    start_date = dt.date(start_date.year, start_date.month, 1)
    
    # Set the end date to the end of the latest month in the date
    end_date = dt.date(end_date.year, end_date.month, calendar.monthrange(end_date.year, end_date.month)[1])
    
    # Create a dataframe containing the entire daterange between the start and end dates
    df = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date))
    
    # Merge the series into the dataframe of dates
    df = df.merge(right=series, how='left', left_index=True, right_index=True)
    
    # Forward-fill the NA values
    df = df.fillna(method='ffill')
    
    # In case there are any NA vaulues remaining, backfill them
    df = df.fillna(method='bfill')
    
    # Calculate each of the differences between dates
    diff_ranges = [1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 28, 180, 365]
    for d in diff_ranges:
        shifted_df = pd.concat((df.value, df.value.shift(d).rename('shifted_value')), axis=1)
        shifted_df.loc[:, f'{d}day_val_ch'] = shifted_df.loc[:, 'value'] - shifted_df.loc[:, 'shifted_value']
        shifted_df.loc[:, f'{d}day_per_ch'] = shifted_df.loc[:, f'{d}day_val_ch'] / shifted_df.loc[:, 'value']
        shifted_df = shifted_df[[f'{d}day_val_ch', f'{d}day_per_ch']]
        df = df.merge(right=shifted_df, how='left', left_index=True, right_index=True)
    
    # Create the month grouper for the dataframe
    m_grouper = df.groupby(pd.Grouper(freq='M'))
    
    # Create the result dataframe, which will be a dataframe containing each month and its relevant calculated values.
    # Start out by calculating the max values
    result_df = m_grouper.max()
    result_df.columns = [str(col) + '_max' for col in result_df.columns]

    # Calculate and join in min values
    calculated_values = m_grouper.min()
    calculated_values.columns = [str(col) + '_min' for col in calculated_values.columns]
    result_df = result_df.merge(right=calculated_values, how='left', left_index=True, right_index=True)
    
    # Calculate and join in mean values
    calculated_values = m_grouper.mean()
    calculated_values.columns = [str(col) + '_mean' for col in calculated_values.columns]
    result_df = result_df.merge(right=calculated_values, how='left', left_index=True, right_index=True)
    
    # Calculate and join in median values
    calculated_values = m_grouper.median()
    calculated_values.columns = [str(col) + '_median' for col in calculated_values.columns]
    result_df = result_df.merge(right=calculated_values, how='left', left_index=True, right_index=True)
    
    # Calculate and join in standard deviation of values
    calculated_values = m_grouper.std()
    calculated_values.columns = [str(col) + '_std' for col in calculated_values.columns]
    result_df = result_df.merge(right=calculated_values, how='left', left_index=True, right_index=True)
    
    # Add the name of the series to each column name
    result_df.columns = [series_name + '_' + str(col) for col in result_df.columns]
    
    return result_df

Next, a similar function is defined to convert weekly data to monthly data. This is performed nearly identically to daily data, involving both calculation of changes from prior weeks and subsequent resampling to monthly granularity.

In [5]:
def weekly_to_monthly(series, series_name):
    """Take in a weekly dataset, and return a monthly df of statistics"""
    # Sort the series by date and turn it into a dataframe
    df = series.sort_index()
    
    # Forward-fill any NA values
    df = df.fillna(method='ffill')
    
    # In case there are any NA vaulues remaining, backfill them
    df = df.fillna(method='bfill')
    
    # Calculate each of the differences between weeks
    diff_ranges = [1, 2, 3, 4, 8, 16, 52]
    for w in diff_ranges:
        shifted_df = pd.concat((df.value, df.value.shift(w).rename('shifted_value')), axis=1)
        shifted_df.loc[:, f'{w}wk_val_ch'] = shifted_df.loc[:, 'value'] - shifted_df.loc[:, 'shifted_value']
        shifted_df.loc[:, f'{w}wk_per_ch'] = shifted_df.loc[:, f'{w}wk_val_ch'] / shifted_df.loc[:, 'value']
        shifted_df = shifted_df[[f'{w}wk_val_ch', f'{w}wk_per_ch']]
        df = df.merge(right=shifted_df, how='left', left_index=True, right_index=True)

    # Create the month grouper for the dataframe
    m_grouper = df.groupby(pd.Grouper(freq='M'))
    
    # Create the result dataframe, which will be a dataframe containing each month and its relevant calculated values.
    # Start out by calculating the max values
    result_df = m_grouper.max()
    result_df.columns = [str(col) + '_max' for col in result_df.columns]

    # Calculate and join in min values
    calculated_values = m_grouper.min()
    calculated_values.columns = [str(col) + '_min' for col in calculated_values.columns]
    result_df = result_df.merge(right=calculated_values, how='left', left_index=True, right_index=True)
    
    # Calculate and join in mean values
    calculated_values = m_grouper.mean()
    calculated_values.columns = [str(col) + '_mean' for col in calculated_values.columns]
    result_df = result_df.merge(right=calculated_values, how='left', left_index=True, right_index=True)
    
    # Calculate and join in median values
    calculated_values = m_grouper.median()
    calculated_values.columns = [str(col) + '_median' for col in calculated_values.columns]
    result_df = result_df.merge(right=calculated_values, how='left', left_index=True, right_index=True)
    
    # Calculate and join in standard deviation of values
    calculated_values = m_grouper.std()
    calculated_values.columns = [str(col) + '_std' for col in calculated_values.columns]
    result_df = result_df.merge(right=calculated_values, how='left', left_index=True, right_index=True)
    
    # Add the name of the series to each column name
    result_df.columns = [series_name + '_' + str(col) for col in result_df.columns]
    return result_df

Next, a function defining how to process monthly data is defined. Monthly data requires far fewer calculations, as no resampling is required - only filling missing values is performed.

In [6]:
def process_monthly(series, series_name):
    series.index = [dt.date(i.year, i.month, calendar.monthrange(i.year, i.month)[1]) for i in series.index]
    
    df = pd.DataFrame(series)
    
    # Forward-fill the NA values
    df = df.fillna(method='ffill')
    
    # In case there are any NA vaulues remaining, backfill them
    df = df.fillna(method='bfill')
    
    diff_ranges = [1, 2, 3, 6, 9, 12]
    for w in diff_ranges:
        shifted_df = pd.concat((df.value, df.value.shift(w).rename('shifted_value')), axis=1)
        shifted_df.loc[:, f'{w}mth_val_ch'] = shifted_df.loc[:, 'value'] - shifted_df.loc[:, 'shifted_value']
        shifted_df.loc[:, f'{w}mth_per_ch'] = shifted_df.loc[:, f'{w}mth_val_ch'] / shifted_df.loc[:, 'value']
        shifted_df = shifted_df[[f'{w}mth_val_ch', f'{w}mth_per_ch']]
        df = df.merge(right=shifted_df, how='left', left_index=True, right_index=True)
    
    df.columns = [series_name + '_' + str(col) for col in df.columns]
    
    return df

The next two functions define how quarterly and annual data are converted to monthly data. Both follow the same process - simply using the date of the quarter or year as the value for the month that it takes place, and imputing each missing month. A variety of imputation methods are included.

In [7]:
def quarterly_to_monthly(series, series_name):
    series.index = [dt.date(i.year, i.month, calendar.monthrange(i.year, i.month)[1]) for i in series.index]
    
    start_date = series.index.min()
    start_date = dt.date(start_date.year, start_date.month, 1)
    end_date = series.index.max()

    # For quarterly data, the data spans through two months after the latest month shown. Add two months to the end date accordingly.
    end_date = end_date + dt.timedelta(days=64)
    end_date = dt.date(end_date.year, end_date.month, 1)

    # Create a dataframe containing the entire daterange between the start and end dates
    df = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='M'))

    # Merge the series into the dataframe of dates
    df = df.merge(right=pd.DataFrame(series), how='left', left_index=True, right_index=True)

    df.columns = ['target']
    
    df[f'interp_linear_{series_name}'] = df.target.interpolate(method='linear')
    df[f'interp_quad_{series_name}'] = df.target.interpolate(method='quadratic')
    df[f'interp_cubic_{series_name}'] = df.target.interpolate(method='cubic')
    df[f'interp_slinear_{series_name}'] = df.target.interpolate(method='slinear')
    df[f'interp_akima_{series_name}'] = df.target.interpolate(method='akima')
    df[f'interp_poly5_{series_name}'] = df.target.interpolate(method='polynomial', order=5)
    df[f'interp_poly7_{series_name}'] = df.target.interpolate(method='polynomial', order=7)

    df = df.rename(columns={'target': 'value'})
        
    diff_ranges = [1, 2, 3, 6, 9, 12]
    for w in diff_ranges:
        shifted_df = pd.concat((df.value, df.value.shift(w).rename('shifted_value')), axis=1)
        shifted_df.loc[:, f'{w}mth_val_ch'] = shifted_df.loc[:, 'value'] - shifted_df.loc[:, 'shifted_value']
        shifted_df.loc[:, f'{w}mth_per_ch'] = shifted_df.loc[:, f'{w}mth_val_ch'] / shifted_df.loc[:, 'value']
        shifted_df = shifted_df[[f'{w}mth_val_ch', f'{w}mth_per_ch']]
        df = df.merge(right=shifted_df, how='left', left_index=True, right_index=True)
    

    
    # Forward-fill the NA values
    df = df.fillna(method='ffill')
    
    # In case there are any NA vaulues remaining, backfill them
    df = df.fillna(method='bfill')
    
    df.columns = [series_name + '_' + str(col) for col in df.columns]
    
    return df

In [8]:
def annual_to_monthly(series, series_name):
    series.index = [dt.date(i.year, i.month, calendar.monthrange(i.year, i.month)[1]) for i in series.index]
    
    start_date = series.index.min()
    start_date = dt.date(start_date.year, start_date.month, 1)
    end_date = series.index.max()

    # For yearly data, the data spans through eleven months after the latest month shown. Add eleven months to the end date accordingly.
    end_date = end_date + dt.timedelta(days=360)
    end_date = dt.date(end_date.year, end_date.month, 1)

    # Create a dataframe containing the entire daterange between the start and end dates
    df = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='M'))

    # Merge the series into the dataframe of dates
    df = df.merge(right=pd.DataFrame(series), how='left', left_index=True, right_index=True)

    df.columns = ['target']
    
    df[f'interp_linear_{series_name}'] = df.target.interpolate(method='linear')
    df[f'interp_quad_{series_name}'] = df.target.interpolate(method='quadratic')
    df[f'interp_cubic_{series_name}'] = df.target.interpolate(method='cubic')
    df[f'interp_slinear_{series_name}'] = df.target.interpolate(method='slinear')
    df[f'interp_akima_{series_name}'] = df.target.interpolate(method='akima')
    df[f'interp_poly5_{series_name}'] = df.target.interpolate(method='polynomial', order=5)
    df[f'interp_poly7_{series_name}'] = df.target.interpolate(method='polynomial', order=7)

    df = df.rename(columns={'target': 'value'})

    diff_ranges = [1, 2, 3, 6, 9, 12]
    for w in diff_ranges:
        shifted_df = pd.concat((df.value, df.value.shift(w).rename('shifted_value')), axis=1)
        shifted_df.loc[:, f'{w}mth_val_ch'] = shifted_df.loc[:, 'value'] - shifted_df.loc[:, 'shifted_value']
        shifted_df.loc[:, f'{w}mth_per_ch'] = shifted_df.loc[:, f'{w}mth_val_ch'] / shifted_df.loc[:, 'value']
        shifted_df = shifted_df[[f'{w}mth_val_ch', f'{w}mth_per_ch']]
        df = df.merge(right=shifted_df, how='left', left_index=True, right_index=True)
    
    # Forward-fill the NA values
    df = df.fillna(method='ffill')
    
    # In case there are any NA vaulues remaining, backfill them
    df = df.fillna(method='bfill')
    
    df.columns = [series_name + '_' + str(col) for col in df.columns]
    
    return df

### Pulling the Data

Each of the FRED series that was enumerated in the s_list above is pulled using the FRED API. Depending on the data's frequency, it is passed into the appropriate function defined above.

Each series is then lagged so that the data would not have been known at the time for which the independent variable is being calculated, combined into one dataframe, and exported.

First, each series is run through the functions defined above according to its frequency. Each function returns a dataframe, and each returned dataframe is put into a dictionary. The dataframes will later be combined into one large dataframe of independent variables.

In [9]:
df_dict = {}
for s in s_list:
    print(f'starting on {s}...')
    # Occasionally, there is a communication error with the server that the requests are made to. The code below is set up to   
    j = 0
    while j < 5:
        try:
            current_series = fred.get_series(s['code']).to_frame()
            current_series.columns = ['value']
            if s['freq'] == 'd':
                df_dict[s['code']] = daily_to_monthly(current_series, s['code'])
            elif s['freq'] == 'w':
                df_dict[s['code']] = weekly_to_monthly(current_series, s['code'])
            elif s['freq'] == 'm':
                df_dict[s['code']] = process_monthly(current_series, s['code'])
            elif s['freq'] == 'q':
                df_dict[s['code']] = quarterly_to_monthly(current_series, s['code'])
            elif s['freq'] == 'a':
                df_dict[s['code']] = annual_to_monthly(current_series, s['code'])
        except:
            print(f"error: {s['code']} not found")
            j += 1
            time.sleep(10)
            assert j < 5, "5 iterations tried on {s['code']}"
            continue
        j = 5
        

starting on {'name': 'Ten Year Minus Two Year Yield Diff', 'code': 'T10Y2Y', 'freq': 'd'}...
starting on {'name': 'Ten Year Minus Three Month Yield Diff', 'code': 'T10Y3M', 'freq': 'd'}...
starting on {'name': 'CPI - All Items', 'code': 'CPIAUCSL', 'freq': 'm'}...
starting on {'name': 'Civilian Unemployment Rate', 'code': 'UNRATE', 'freq': 'm'}...
starting on {'name': 'US RGDP', 'code': 'GDPC1', 'freq': 'q'}...
starting on {'name': '30-Year Fixed Mortgage Average', 'code': 'MORTGAGE30US', 'freq': 'w'}...
starting on {'name': 'US NGDP', 'code': 'GDP', 'freq': 'q'}...
error: GDP not found
starting on {'name': 'Effective Federal Funds Rate', 'code': 'FEDFUNDS', 'freq': 'm'}...
starting on {'name': 'Ten Year Treasury Constant Maturity Rate', 'code': 'DGS10', 'freq': 'd'}...
starting on {'name': 'ICE BofAML US High Yield Master II Option-Adjusted Spread', 'code': 'BAMLH0A0HYM2', 'freq': 'd'}...
starting on {'name': 'S&P/Case-Shiller U.S. National Home Price Index', 'code': 'CSUSHPINSA', 'fr

starting on {'name': 'Chicago Fed National Financial Conditions Index', 'code': 'NFCI', 'freq': 'w'}...
starting on {'name': 'Unemployment Rate: 20 years and over, Black or African American Men', 'code': 'LNS14000031', 'freq': 'm'}...
starting on {'name': 'Nonfinancial corporate business; debt securities; liability, Level', 'code': 'NCBDBIQ027S', 'freq': 'q'}...
starting on {'name': 'Capacity Utilization: Total Industry', 'code': 'TCU', 'freq': 'm'}...
starting on {'name': 'ICE BofAML US High Yield BB Effective Yield', 'code': 'BAMLH0A1HYBBEY', 'freq': 'd'}...
starting on {'name': 'Average Sales Price of Houses Sold for the United States', 'code': 'ASPUS', 'freq': 'q'}...
starting on {'name': 'Real Disposable Personal Income: Per Capita', 'code': 'A229RX0', 'freq': 'm'}...
starting on {'name': 'Unemployment Level', 'code': 'UNEMPLOY', 'freq': 'm'}...
starting on {'name': 'Homeownership Rate for the United States', 'code': 'RHORUSQ156N', 'freq': 'q'}...
starting on {'name': 'Motor Vehic

Next, the dependent variable - the default rate of credit card loans - is pulled and resampled to monthly.

In [10]:
dep_series = fred.get_series('DRCCLACBN')

In [11]:
# Set the dependent series index to the last day in the month to correspond to the rest of the datasets
dep_series.index = [dt.date(i.year, i.month, calendar.monthrange(i.year, i.month)[1]) for i in dep_series.index]

# Determine the start and end dates for the series, which will be used later on.
start_date = dep_series.index.min()
start_date = dt.date(start_date.year, start_date.month, 1)
end_date = dep_series.index.max()

# For quarterly data, the data spans through two months after the latest month shown. Add two months to the end date accordingly.
end_date = end_date + dt.timedelta(days=64)
end_date = dt.date(end_date.year, end_date.month, 1)

# Create a dataframe containing the entire daterange between the start and end dates
dep_df = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='M'))

# Merge the series into the dataframe of dates
dep_df = dep_df.merge(right=pd.DataFrame(dep_series), how='left', left_index=True, right_index=True)

# Create a copy of the dependent data pre-imputation. This will be used later on in the project.
dep_df_raw = dep_df.copy()

# Interpolate the data using the quadratic method, which seems to best fit the expected path of defaults
dep_df.columns = ['target']
dep_df[f'interp_quad'] = dep_df.target.interpolate(method='quadratic')
dep_df = dep_df.ffill()

dep_df = dep_df[['interp_quad']]

Lags for each variable in the dataframe are now added, as it is possible that variables from prior months have an influence on the dependent variable. For example, each row is modified to contain both the variable values from the month that the row represents, and the variable values from the prior month.

The dataframes are then combined.

In [16]:
ind_df = pd.DataFrame(index=dep_df.index)

for k, series_df in df_dict.items():
    ind_df = pd.merge(left=ind_df, right=series_df, left_index=True, right_index=True, how='left')

Finally, the independent and dependent data are exported to pickle files for future processing.

In [17]:
ind_df.to_pickle(r'data\ind_df.pkl')
dep_df.to_pickle(r'data\dep_df.pkl')
dep_df_raw.to_pickle(r'data\dep_df_raw.pkl')